In [1]:
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

UsageError: Line magic function `%` not found.


In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')

clear_output()

In [3]:
#import Flair Classifier
from flair.models import TextClassifier

classifier = TextClassifier.load('en-sentiment')

clear_output()

In [4]:
#@title Enter Twitter Credentials
TWITTER_KEY = 'HTi8RVQq3LczryTEyESQBdqNk' #@param {type:"string"}
TWITTER_SECRET_KEY = '5VYQAWZTLuBs2p9mbLmkoGvCg7iR0vnXvKMdEDyEGcSKS5jcQk' #@param {type:"string"}

In [5]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [6]:
#@title Twitter Search API Inputs
#@markdown ### Enter Search Query:
searchQuery = 'Pakistan' #@param {type:"string"}
#@markdown ### Enter Max Tweets To Scrape:
#@markdown #### The Twitter API Rate Limit (currently) is 45,000 tweets every 15 minutes.
maxTweets = 100 #@param {type:"slider", min:0, max:45000, step:100}
Filter_Retweets = True #@param {type:"boolean"}

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()

    
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.user.followers_count, tweet.text, tweet.favorite_count, 
                    reply_count, tweet.retweet_count, retweeted, tweet.user.created_at, tweet.user.verified, tweet.user.location, tweet.user.statuses_count])
        
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break


print("Downloaded {0} tweets".format(tweetCount))



Downloaded 100 tweets
Downloaded 100 tweets


In [7]:
pd.set_option('display.max_colwidth', None)

# load it into a pandas dataframe
df_p = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'followers', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted', 'account_start_date', 'verified_user', 'location', 'number_of_tweets'])

df_p.head(10)

,tweet_dt,topic,id,username,name,followers,tweet,like_count,reply_count,retweet_count,retweeted,account_start_date,verified_user,location,number_of_tweets
0,2020-09-18,Pakistan,1306813944862998532,GujjarJobs,Nadeem Anjum Gujjar,157,The Khyber Pass links which two countries?\nAfghanistan and Pakistan,0,0,0,False,2012-03-13 07:40:41,False,Kot Gujjran Multan,2958
1,2020-09-18,Pakistan,1306813944632242176,HBaarot,Hiren Barot,0,@IMRANKH51363466 @ReallySwara Terror means Islam. Horrible. They marriage with their sisters. Rediculous. They sta… https://t.co/YFkcg3ux6e,0,0,0,False,2017-11-25 12:06:45,False,,91
2,2020-09-18,Pakistan,1306813943285862401,krab09,Prasad Karambelkar,46,"Suddenly everyone wants to nonch India. Nepal came up with new Maps, Pakistan doing the same. We can very well conn… https://t.co/PiBA65B2QX",0,0,0,False,2009-10-24 11:01:40,False,"Bengaluru, India",4460
3,2020-09-18,Pakistan,1306813879641542656,pardhu_leo,NRC & UCC Next 🇮🇳 Jai Shri Ram🚩,2072,Entire humanity except #Pakistan #China https://t.co/BzAX0rkJDS,0,0,0,False,2013-08-13 06:38:31,False,Bharat,83423
4,2020-09-18,Pakistan,1306813878773395457,abidhussainuk,PTI,230,@FaisalJavedKhan @StateBank_Pak Can you transfer money from existing bank accounts in pakistan into the new roshan digital bank account?,0,0,0,False,2013-03-01 17:44:56,False,UK,10725
5,2020-09-18,Pakistan,1306813850919088130,BhambriKastura,Kastura Bhambri,705,A country whr terrorists carrying guns can roam more freely than women on streets!\nWelcm 2 Islamic Republic of Paki… https://t.co/vtG33vN53n,0,0,0,False,2020-01-22 04:20:56,False,"India, Los Angeles",664
6,2020-09-18,Pakistan,1306813841595006976,KabirMohit,Mohit Pandey,344,"The PM seemed to have taken the luxury of having Pakistan at hand for granted, and ignored the fact that the nation… https://t.co/KSqliQqxgA",0,0,0,False,2013-11-14 18:01:01,False,Gaya ✈️ New Delhi ✈️ Gaya,783
7,2020-09-18,Pakistan,1306813838474358784,KabirMohit,Mohit Pandey,344,"The prime minister made an error of judgement when he said, “Hum Bajrang Bali ki jai bolkar upar se chale gaye” whi… https://t.co/qJzenjYqOp",0,0,0,False,2013-11-14 18:01:01,False,Gaya ✈️ New Delhi ✈️ Gaya,783
8,2020-09-18,Pakistan,1306813835790045185,Indic_e,indic 🌧️🌈,69,@krupali @USPS @JoeBiden @USPSHelp @OIGUSPS @RexChapman @dog_rates @DoctorsForBiden @KamalaHarris @meenaharris… https://t.co/FsU0cj1UwU,0,0,0,False,2019-09-06 02:25:26,False,"Ram Kund, Pakistan",31979
9,2020-09-18,Pakistan,1306813833693089793,HarisKhalil10,Haris Khalil,3101,@Shanzayhere It's better to create individual Pakistan,0,0,0,False,2017-03-31 11:33:53,False,Lahore pakistan,58027


In [18]:
#@title Filter By Date Range
today = datetime.now().date()
yesterday = today - timedelta(1)

start_dt = '2020-09-01' #@param {type:"date"}
end_dt = '2020-09-14' #@param {type:"date"}

if start_dt == '':
  start_dt = yesterday
else:
  start_dt = datetime.strptime(start_dt, '%Y-%m-%d').date()

if end_dt == '':
  end_dt = today
else:
  end_dt = datetime.strptime(end_dt, '%Y-%m-%d').date()


df_p = df_p[(df_p['tweet_dt'] >= start_dt) 
                    & (tweet_df['tweet_dt'] <= end_dt)]
df_p.shape

NameError: name 'tweet_df' is not defined

In [23]:
#predict NER
nerlst = []

for index, row in tqdm(df_p.iterrows(), total=df_p.shape[0]):
  cleanedTweet = row['tweet'].replace("#", "")
  sentence = Sentence(cleanedTweet, use_tokenizer=True)
  
  # predict NER tags
  tagger.predict(sentence)

  # get ner
  ners = sentence.to_dict(tag_type='ner')['entities']
  
  # predict sentiment
  classifier.predict(sentence)
  
  label = sentence.labels[0]
  response = {'result': label.value, 'polarity':label.score}
  
  # get hashtags
  hashtags = re.findall(r'#\w+', row['tweet'])
  if len(hashtags) >= 1:
    for hashtag in hashtags:
      ners.append({ 'type': 'Hashtag', 'text': hashtag })
  
  for ner in ners:
    adj_polarity = response['polarity']
    if response['result'] == 'NEGATIVE':
      adj_polarity = response['polarity'] * -1
    try:
      ner['type']
    except:
      ner['type'] = ''      
    nerlst.append([ row['tweet_dt'], row['topic'], row['id'], row['username'], 
                   row['name'], row['followers'], row['tweet'], ner['type'], ner['text'], response['result'], 
                   response['polarity'], adj_polarity, row['like_count'], row['reply_count'], 
                  row['retweet_count'], row['account_start_date'], row['verified_user'], row['location'], row['number_of_tweets'] ])

clear_output()

In [25]:
df_pner = pd.DataFrame(nerlst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'followers', 'tweet', 'tag_type', 'tag', 'sentiment', 'polarity', 
                                      'adj_polarity', 'like_count', 'reply_count', 'retweet_count', 'account_start_date', 'verified_user', 'location', 'number_of_tweets'])



In [26]:
def days_before(date):
  if date and date != 0 or date != '0':
      date = str(date)
      date = date.split()[0]
      now = str(datetime.now())
      now = now.split()[0]
      d1 = datetime.strptime(date, "%Y-%m-%d")
      d2 = datetime.strptime(now, "%Y-%m-%d")
      return abs((d2 - d1).days)
  else:
    return ''

In [32]:
#This cell does main data analysis
df_pner2 = df_p
#Removes all the duplicate tweets
df_pner2 = df_pner.drop_duplicates(subset = 'id')
df_pner2 = df_pner.reset_index(drop = True)
del df_pner2['polarity']
#df_pner2['mean polarity'] = df_p['adj_polarity'].mean()
#df_pner2['polarity standard deviation'] = df_p['adj_polarity'].std(axis = 0, ddof = 0)
#df_pner2['average daily tweets'] = 0
#df_pner2 = df_ner_1[['tweet_dt',	'topic',	'id',	'username',	'name',	'followers',	'tweet',	'tag', 'tag_type',	'average daily tweets', 'sentiment', 'adj_polarity',	'mean polarity',	'polarity standard deviation', 'like_count',	'reply_count',	'retweet_count', 'account_start_date', 'verified_user', 'location', 'number_of_tweets']]


#df_pner2['account_start_date'] = df_ner_1['account_start_date'].apply(days_before)
#df_pner2['average daily tweets'] = df_ner_1['number_of_tweets']/df_ner_1['account_start_date']
#del df_pner2['account_start_date']

df_pner2.rename(columns = {'followers': 'number of followers', 'adj_polarity': 'polarity', 'number_of_tweets': 'total number of tweets'}, inplace = True)
# df_ner_1 = df_ner_1.sort_values(by = 'average daily tweets', ascending=False)
df_pner2.tail()

,tweet_dt,topic,id,username,name,number of followers,tweet,tag_type,tag,sentiment,polarity,like_count,reply_count,retweet_count,account_start_date,verified_user,location,total number of tweets
222,2020-09-18,Pakistan,1306812319708528641,mohandixit,INDIAN. HINDUSTANI,86,@RvUpadhya @dineshgrao He will not talk about Chinese virus or Chinese aggression. Or about PAKISTAN TERRORISM beca… https://t.co/YsCNc9ljIP,,PAKISTAN,NEGATIVE,-0.993165,0,0,0,2011-11-17 03:29:40,False,,14280
223,2020-09-18,Pakistan,1306812294853165057,ImranInc,Imran Ahmad,5451,PMIK address at the 4th Annual Austrian World Summit explains how Pakistan took revolutionary steps through plantin… https://t.co/QNK3tIaEbs,,PMIK,POSITIVE,0.723712,1,0,1,2011-10-30 15:34:31,False,,40869
224,2020-09-18,Pakistan,1306812294853165057,ImranInc,Imran Ahmad,5451,PMIK address at the 4th Annual Austrian World Summit explains how Pakistan took revolutionary steps through plantin… https://t.co/QNK3tIaEbs,,4th Annual,POSITIVE,0.723712,1,0,1,2011-10-30 15:34:31,False,,40869
225,2020-09-18,Pakistan,1306812294853165057,ImranInc,Imran Ahmad,5451,PMIK address at the 4th Annual Austrian World Summit explains how Pakistan took revolutionary steps through plantin… https://t.co/QNK3tIaEbs,,Austrian World Summit,POSITIVE,0.723712,1,0,1,2011-10-30 15:34:31,False,,40869
226,2020-09-18,Pakistan,1306812294853165057,ImranInc,Imran Ahmad,5451,PMIK address at the 4th Annual Austrian World Summit explains how Pakistan took revolutionary steps through plantin… https://t.co/QNK3tIaEbs,,Pakistan,POSITIVE,0.723712,1,0,1,2011-10-30 15:34:31,False,,40869


In [33]:
#Saves as csv
file_name = 'pakistan_data.csv'
df_pner2.to_csv(file_name, index = False)